In [1]:
import numpy
import numpy as np
import pandas as pd
import sys, glob
import urllib3
import h5py
import cdsapi, zipfile, os, time
import warnings
import shutil
import xarray
from datetime import date
warnings.filterwarnings('ignore')
# import pycountry
sys.path.append(os.getcwd()+'/../cds-backend/code/')
import cds_eua3 as eua
import numba
import copy
import glob
from numba import njit
import pandas
import glob
import multiprocessing
from functools import partial

In [2]:
c = cdsapi.Client()
r = c.retrieve('insitu-comprehensive-upper-air-observation-network',
               {
                   'variable': ["temperature"],
                   'date': ['20001229','20151230'],
                   'statid': '10393',
#                    'pressure_level':  ['10','20','30','50'],
                   'optional': ['desroziers_30','desroziers_60', 'desroziers_90', 'desroziers_180']
               })
r.download(target='download.zip')
assert os.stat('download.zip').st_size == r.content_length, "Downloaded file is incomplete"
z = zipfile.ZipFile('download.zip')
z.extractall(path='./')
z.close()
cdsfile = glob.glob('./*10393*.nc')

2021-07-21 09:25:53,514 INFO Welcome to the CDS
2021-07-21 09:25:53,515 INFO Sending request to https://sis-dev.climate.copernicus.eu/api/v2/resources/insitu-comprehensive-upper-air-observation-network
2021-07-21 09:25:53,752 INFO Request is completed
2021-07-21 09:25:53,753 INFO Downloading http://136.156.132.176/cache-compute-0000/cache/data2/adaptor.comprehensive_upper_air.retrieve-1626796615.6664505-17472-16-1898f899-94fc-4a6e-ab0a-469fb4bef8cc.zip to download.zip (221.4M)
2021-07-21 09:26:20,114 INFO Download rate 8.4M/s  


In [3]:
c = cdsapi.Client()
r = c.retrieve('insitu-comprehensive-upper-air-observation-network',
               {
                   'variable': ["temperature"],
                   'date': ['20001229','20151230'],
                   'statid': '11035',
#                    'pressure_level':  ['10','20','30','50'],
                   'optional': ['desroziers_30','desroziers_60', 'desroziers_90', 'desroziers_180']
               })
r.download(target='download.zip')
assert os.stat('download.zip').st_size == r.content_length, "Downloaded file is incomplete"
z = zipfile.ZipFile('download.zip')
z.extractall(path='./')
z.close()
cdsfile = glob.glob('./*11035*.nc')

2021-07-21 09:26:21,280 INFO Welcome to the CDS
2021-07-21 09:26:21,281 INFO Sending request to https://sis-dev.climate.copernicus.eu/api/v2/resources/insitu-comprehensive-upper-air-observation-network
2021-07-21 09:26:21,498 INFO Request is completed
2021-07-21 09:26:21,536 INFO Downloading http://136.156.132.176/cache-compute-0000/cache/data2/adaptor.comprehensive_upper_air.retrieve-1626796641.628392-21059-15-0462ec42-ef26-44b6-976f-44777b343438.zip to download.zip (40.8M)
2021-07-21 09:26:26,247 INFO Download rate 8.7M/s   


In [4]:
lind = glob.glob('./*10393*.nc')[0]
vie = glob.glob('./*11035*.nc')[0]

---

In [6]:
with h5py.File('./merge.nc') as merge:
    name = ['vie', 'lind']
    file = [vie, lind]
    for j in range(len(name)):
            merge.create_group(name[j])
            with h5py.File(file[j], 'r') as filetocopy:
                for i in filetocopy.keys():
                    print(i)
                    merge[name[j]].create_dataset_like(i, filetocopy[i], chunks=True)
                    merge[name[j]][i][:] = filetocopy[i][:]
                    for a in filetocopy[i].attrs.keys():
                        if a not in ['DIMENSION_LIST', 'CLASS', 'external_table']:
                            if type(filetocopy[i].attrs[a]) is str:
                                merge[name[j]][i].attrs[a] = np.string_(filetocopy[i].attrs[a])
                            else:
                                merge[name[j]][i].attrs[a] = filetocopy[i].attrs[a]

desroziers_180
desroziers_30
desroziers_60
desroziers_90
lat
lon
obs
plev
report_id
station_id
string11
string25
ta
time
trajectory
trajectory_index
trajectory_label
desroziers_180
desroziers_30
desroziers_60
desroziers_90
lat
lon
obs
plev
report_id
station_id
string11
string20
ta
time
trajectory
trajectory_index
trajectory_label


---

In [7]:
with h5py.File('./merge.nc', 'r') as data:
    print(data)
    print(data.keys())
    for i in data.keys():
        print(i)
        print('---')
        for j in data[i].keys():
            print(j, len(data[i][j][:]))
            print(data[i][j].attrs.keys())
            print()
        
        print('\n')

<HDF5 file "merge.nc" (mode r)>
<KeysViewHDF5 ['lind', 'vie']>
lind
---
desroziers_180 7120490
<KeysViewHDF5 ['cdmname', 'coordinates', 'description', 'units']>

desroziers_30 7120490
<KeysViewHDF5 ['cdmname', 'coordinates', 'description', 'units']>

desroziers_60 7120490
<KeysViewHDF5 ['cdmname', 'coordinates', 'description', 'units']>

desroziers_90 7120490
<KeysViewHDF5 ['cdmname', 'coordinates', 'description', 'units']>

lat 7120490
<KeysViewHDF5 ['cdmname', 'standard_name', 'units']>

lon 7120490
<KeysViewHDF5 ['cdmname', 'standard_name', 'units']>

obs 7120490
<KeysViewHDF5 ['NAME']>

plev 7120490
<KeysViewHDF5 ['cdmname', 'standard_name', 'units']>

report_id 7120490
<KeysViewHDF5 ['cdmname', 'coordinates', 'units']>

station_id 7120490
<KeysViewHDF5 ['cdmname', 'coordinates', 'standard_name', 'units']>

string11 11
<KeysViewHDF5 ['NAME']>

string20 20
<KeysViewHDF5 ['NAME']>

ta 7120490
<KeysViewHDF5 ['cdmname', 'cdsname', 'coordinates', 'standard_name', 'units']>

time 7120490

---

## parallel access

In [8]:
def testfunc(file, group):
    with h5py.File(file, 'r') as data:
        print(data[group]['ta'][:])
    return 0

stations = ['vie', 'lind']
file = './merge.nc'
pool = multiprocessing.Pool(processes=2)
func=partial(testfunc, file)
result_list = list(pool.map(func, stations))

[222.7  221.5  217.1  ... 270.39 270.5  270.65]
[220.5  212.5  203.9  ... 269.85 269.84 269.77]


In [9]:
def testfunc(file, group):
    with h5py.File(file, 'r') as data:
        for i in data[group]['ta'][:100]:
            print(group, i)
    return 0

stations = ['vie', 'lind']
file = './merge.nc'
pool = multiprocessing.Pool(processes=2)
func=partial(testfunc, file)
result_list = list(pool.map(func, stations))

lind 220.5
lind 212.5
vielind  203.9
lind222.7 205.5

lind vie210.1
lind 209.5 
lind 221.5208.1
lind
 213.3
lindvie 215.1
lind  215.1
lind217.1 215.3

lind 218.1vie
lind  217.7
lind218.1 220.5

lind vie218.9
 lind215.5 
218.5
vielind  214.5219.3

lind vie219.5 
207.7lind 
216.9vie
lind  206.5211.9

lindvie  213.1
lind210.3 213.5

lind vie214.1
lind  218.9211.7
lind 
219.7vie
 lind 211.7222.9

lindvie 225.5 
lind 208.3235.5

lind vie236.1
lind  239.3209.7
lind
 vie243.9
 lind 211.7246.3
lind
 246.7vie
lind  248.1
210.3lind 249.3

lindvie  249.9211.5
lind
 251.5vie
lind  253.7
216.3lind 253.9

lind vie254.7
lind  254.3
214.5lind 254.3

lind vie257.9
 lind 260.1213.9
lind 
263.7
lindvie  267.7213.9
lind
 267.9
vielind  268.1
216.3lind 
269.9
lindvie  271.1
216.9lind 
271.5
vielind 271.7 
lind214.5 270.9

lind vie223.7 
lind 214.3218.7

lind vie217.5 
lind217.1 211.3

lindvie 210.9 
lind216.7 205.9

lind vie210.1
lind  209.5220.1
lind 
208.9
lindvie 213.7
 lind217.7 
211.7
vielind  211.3
l